In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

In [3]:
import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [4]:
data = np.load('/home/ML4NO/Data/n1000000_0910_classification.npz')

In [5]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
target = data['cpv']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [6]:
def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))  
    model_DNN.add(keras.Input(shape=(len(x_train[0]),), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_2'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_3'))
    model_DNN.add(Dense(1, activation='sigmoid', name = 'output'))
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss="binary_crossentropy",
                              optimizer=model_opt,
                              metrics=['accuracy'])
    model_DNN.summary()
    return model_DNN

In [7]:
model = DNN_Model("classification")

Model: "Model_DNN_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 264)               1056      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               135680    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
output (Dense)               (None, 1)                 513       
Total params: 662,561
Trainable params: 662,033
Non-trainable params: 528
_________________________________________________________________


In [9]:
from tensorflow import keras 
from tensorflow.keras.preprocessing import image
from keras.utils.vis_utils import plot_model
plot_model(model)

ImportError: cannot import name 'get_config'

In [18]:
model_index = 1
while os.path.isfile('/home/ML4NO/ML_DUNE/Classification/models_all/models/0910_{}.h5'.format(model_index)):
    model_index += 1

In [16]:
log_dir = "logs/all/{}".format(model_index)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train2, y_train2,
           validation_split = 0.1,
           callbacks=[tensorboard_callback],
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 71s 6ms/step - loss: 0.2350 - accuracy: 0.9136 - val_loss: 0.1616 - val_accuracy: 0.9430
Epoch 2/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1911 - accuracy: 0.9335 - val_loss: 0.1639 - val_accuracy: 0.9442
Epoch 3/20
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1863 - accuracy: 0.9356 - val_loss: 0.1552 - val_accuracy: 0.9475
Epoch 4/20
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1833 - accuracy: 0.9368 - val_loss: 0.1535 - val_accuracy: 0.9472
Epoch 5/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1834 - accuracy: 0.9371 - val_loss: 0.1488 - val_accuracy: 0.9492
Epoch 6/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1824 - accuracy: 0.9374 - val_loss: 0.1467 - val_accuracy: 0.9499
Epoch 7/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1807 - accuracy: 0.9379 - val_loss: 0

In [19]:
model.save('/home/ML4NO/ML_DUNE/Classification/models_all/models/0910_{}.h5'.format(model_index))

In [20]:
furthur_index = 1
path = '/home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_{}_{}'
while os.path.isdir(path.format(model_index, furthur_index)):
    furthur_index += 1

In [21]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    log_dir = "logs/all/{}_{}(std={})".format(model_index, furthur_index, scale)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               callbacks=[tensorboard_callback],
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])
    model.save(path.format(model_index, furthur_index) + '/std={}'.format(scale))

3125/3125 [==============================] - 9s 3ms/step - loss: 0.1496 - accuracy: 0.9485
Epoch 1/5
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1778 - accuracy: 0.9388 - val_loss: 0.1389 - val_accuracy: 0.9527
Epoch 2/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1767 - accuracy: 0.9396 - val_loss: 0.1418 - val_accuracy: 0.9529
Epoch 3/5
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1770 - accuracy: 0.9399 - val_loss: 0.1470 - val_accuracy: 0.9504
Epoch 4/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1766 - accuracy: 0.9400 - val_loss: 0.1543 - val_accuracy: 0.9469
Epoch 5/5
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1398 - accuracy: 0.9519

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_1_1/std=0.001/assets


INFO:tensorflow:Assets written to: /home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_1_1/std=0.001/assets


3125/3125 [==============================] - 9s 3ms/step - loss: 0.1398 - accuracy: 0.9519
Epoch 1/5
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1755 - accuracy: 0.9401 - val_loss: 0.1415 - val_accuracy: 0.9522
Epoch 2/5
12516/12516 [==============================] - 69s 6ms/step - loss: 0.1767 - accuracy: 0.9400 - val_loss: 0.1469 - val_accuracy: 0.9500
Epoch 3/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1749 - accuracy: 0.9406 - val_loss: 0.1517 - val_accuracy: 0.9488
Epoch 4/5
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1749 - accuracy: 0.9407 - val_loss: 0.1527 - val_accuracy: 0.9472
Epoch 5/5
3125/3125 [==============================] - 10s 3ms/step - loss: 0.1412 - accuracy: 0.9517

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.

...

INFO:tensorflow:Assets written to: /home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_1_1/std=0.7880462815669912/assets


3125/3125 [==============================] - 10s 3ms/step - loss: 0.3302 - accuracy: 0.8621
Epoch 1/5
12516/12516 [==============================] - 71s 6ms/step - loss: 0.3721 - accuracy: 0.8382 - val_loss: 0.3289 - val_accuracy: 0.8630
Epoch 2/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.3697 - accuracy: 0.8394 - val_loss: 0.3307 - val_accuracy: 0.8627
Epoch 3/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.3690 - accuracy: 0.8395 - val_loss: 0.3194 - val_accuracy: 0.8685
Epoch 4/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.3694 - accuracy: 0.8390 - val_loss: 0.3220 - val_accuracy: 0.8667
Epoch 5/5
3125/3125 [==============================] - 9s 3ms/step - loss: 0.3238 - accuracy: 0.8660

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_1_1/std=1.0/assets


INFO:tensorflow:Assets written to: /home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_1_1/std=1.0/assets


In [ ]:
path = '/home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_{}_{}.h5'
model.save(path.format(model_index, furthur_index))
df = pd.DataFrame({'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss': after_train_loss})
df.to_csv(file = '/home/ML4NO/ML_DUNE/Classification/models_all/models_furthurTrain/0910_{}_{}_result.npy'.format(model_index, index))

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(20):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
Poisson_index = 1
path = '/home/ML4NO/ML_DUNE/Classification/models_all/models_PoissonTrain/0910_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, Poisson_index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))